In [ ]:
from sqlvis import vis

In [ ]:
import sqlite3

students_conn = sqlite3.connect('../../data/QueryVis qualification students dataset.db')
forums_conn = sqlite3.connect('../../data/QueryVis qualification forums dataset.db')
music_conn = sqlite3.connect('../../data/QueryVis main user study DB.db')

students_schema = vis.schema_from_conn(students_conn)
forums_schema = vis.schema_from_conn(forums_conn)
music_schema = vis.schema_from_conn(music_conn)

In [ ]:
students_schema

In [ ]:
forums_schema

In [ ]:
music_schema

# Students questions on Github
[queryvis-mturk Github SQL file link](https://github.com/leventidis/queryvis-mturk/blob/master/static/Qualification%20test.sql)

**Problem 1:**

Show names of courses that 'Student A' Registered for.

In [ ]:
query = '''
SELECT c.Course_Name
FROM Courses c 
    INNER JOIN StudentCourseRegistration r ON c.CourseID = r.CourseID
    INNER JOIN Students s ON s.StudentID = r.StudentID
WHERE Student_Name = 'Student A';
'''

# Generate the visualization.
vis.visualize(query, students_schema)

cur = students_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

In [ ]:
# Should be effectively the same visualization
query = '''
SELECT c.Course_Name
FROM Courses c, StudentCourseRegistration r, Students s 
WHERE Student_Name = 'Student A'
and c.CourseID = r.CourseID
and s.StudentID = r.StudentID;
'''

# Generate the visualization.
vis.visualize(query, students_schema)

cur = students_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)



**Problem 2:**

Show courses with course names and the number of student in each course

In [ ]:
# In standard SQLVis, the COUNT(*) remains curiously invisible in the vis.
query = '''
SELECT Course_Name, COUNT(*)
FROM Courses c
    JOIN StudentCourseRegistration r ON c.CourseID = r.CourseID
GROUP BY r.CourseID;
'''

# Generate the visualization.
vis.visualize(query, students_schema)

cur = students_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

In [ ]:
# SQLVis (or rather, its parser) does not support NATURAL JOIN.
# Query is valid and runs fine though...?
query = '''
SELECT Course_Name, COUNT(*)
FROM Courses c
    NATURAL JOIN StudentCourseRegistration r
GROUP BY r.CourseID;
'''

# Generate the visualization.
vis.visualize(query, students_schema)

cur = students_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

**Problem 3:**

Find students who registered for some course or courses.

In [ ]:
query = '''
SELECT *
FROM Students
WHERE StudentID IN (SELECT StudentID
                    FROM StudentCourseRegistration); 
'''

# Generate the visualization.
vis.visualize(query, students_schema)

cur = students_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

# Forums questions on Github

**Problem 4:**

List posts that were posted after January 1, 2017 0:0:0

In [ ]:
query = '''
SELECT * FROM Posts WHERE Post_DateTime > '2017-01-01 0:0:0';
'''

# Generate the visualization.
vis.visualize(query, forums_schema)

cur = forums_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

**Problem 5:**

List forums that have no post.

In [ ]:
# SQLVis does not show anything about the WHERE NOT EXISTS?
query = '''
SELECT *
FROM Forums
WHERE NOT EXISTS (SELECT *
                  FROM Posts
                  WHERE Post_ForumID = ForumID);
'''

# Generate the visualization.
vis.visualize(query, forums_schema)

cur = forums_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

**Problem 6:**

List all forums by name in which there are at least 2 posts

In [ ]:
# Why is there no visualization at all??
query = '''
SELECT Forum_Name, COUNT(PostID)
FROM Posts
    JOIN Forums ON ForumID = Post_ForumID
GROUP BY Post_ForumID
HAVING COUNT(PostID) >= 2;
'''

# Generate the visualization.
vis.visualize(query, forums_schema)

cur = forums_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

# Qualification Test Questions
As made available on [link](https://osf.io/aw387)

**Question #1**

In [ ]:
query = '''
SELECT P.PlaylistId, P.Name
FROM Playlist P, PlaylistTrack PT, Track T, Album AL, Artist A
WHERE P.PlaylistId = PT.PlaylistId
AND PT.TrackId = T.TrackId
AND T.AlbumId = AL. AlbumId
AND AL.ArtistId = A.ArtistId
AND A.Name = 'AC/DC';
'''

# Generate the visualization.
vis.visualize(query, music_schema)

cur = music_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

**Question 2**

In [ ]:
query = '''
SELECT C.CustomerId, C.FirstName, C.LastName
FROM Customer C, Invoice I, InvoiceLine IL1, InvoiceLine IL2,
     Track T1, Track T2
WHERE C.CustomerId = I.CustomerId
AND I.InvoiceId = IL1.InvoiceId
AND I.InvoiceId = IL2.InvoiceId
AND IL1.TrackId = T1.TrackId
AND IL2.TrackId = T2.TrackId
AND T1.GenreId <> T2.GenreId;
'''

# Generate the visualization.
vis.visualize(query, music_schema)

cur = music_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output (truncated):\n', output[:100])

**Question 3**

In [ ]:
query = '''
SELECT P.PlaylistId, G.Name, COUNT(T.TrackId)
FROM Playlist P, PlaylistTrack PT, Track T, Genre G
WHERE P.PlaylistId = PT.PlaylistId
AND PT.TrackId = T.TrackId
AND T.GenreId = G.GenreId
GROUP BY P.PlaylistId, G.Name;
'''

# Generate the visualization.
vis.visualize(query, music_schema)

cur = music_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

**Question 4**

In [ ]:
query = '''
SELECT A.ArtistId, A.Name
FROM Artist A
WHERE NOT EXISTS (SELECT *
                  FROM Album AL
                  WHERE AL.ArtistId = A.ArtistId
                  AND NOT EXISTS (SELECT *
                                  FROM Track T, MediaType MT
                                  WHERE AL.AlbumId = T.AlbumId
                                  AND T.MediaTypeId = MT.MediaTypeId
                                  AND MT.Name = 'ACC audio file')
                 );
'''

# Generate the visualization.
vis.visualize(query, music_schema)

cur = music_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

**Question 5**

In [ ]:
query = '''
SELECT C1.CustomerId, C1.FirstName, C1.LastName
FROM Customer C1, Invoice I1, InvoiceLine IL1,
     Track T1, Album AL1, Artist A1
WHERE C1.customerId = I1.CustomerId
AND I1.InvoiceId = IL1.InvoiceId
AND IL1.TrackId = T1.TrackId
AND T1.AlbumId = AL1.AlbumId
AND AL1.ArtistId = A1.ArtistId
AND A1.Name = 'AC/DC'
AND NOT EXISTS (SELECT *
                FROM Customer C2, Invoice I2, InvoiceLine IL2,
                     Track T2, Album AL2, Artist A2
                WHERE C2.CustomerId <> C1.CustomerId
                AND C1.City = C2.City
                AND C2.CustomerId = I2.CustomerId
                AND I2.InvoiceId = IL2.InvoiceId
                AND IL2.TrackId = T2.TrackId
                AND T2.AlbumId = AL2.AlbumId
                AND AL2.ArtistId = A2.ArtistId
                AND A2.Name = 'AC/DC');
'''

# Generate the visualization.
vis.visualize(query, music_schema)

cur = music_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)

**Question 6**

In [ ]:
query = '''
SELECT E1.EmployeeId, COUNT(C.CustomerId), AVG(I.Total)
FROM Employee E1, Employee E2, Customer C, Invoice I
WHERE E1.ReportsTo = E2.EmployeeId
AND E1.Country <> E2.Country
AND E1.EmployeeId = C.SupportRepId
AND E1.Country = C.Country
AND C.CustomerId = I.CustomerId
GROUP BY E1.EmployeeId;
'''

# Generate the visualization.
vis.visualize(query, music_schema)

cur = music_conn.cursor()
res = cur.execute(query)
output = res.fetchall()
print('Query output:\n', output)